## 근린분석  
1. 공간 데이터 간의 거리를 산출하는 기법으로 가까운 다른 공간데이터를 찾아 봅시다      
2. 버퍼(Buffer)를 만들고, 버퍼에 포함되는 포인트들을 선택해 봅시다 

### 패키지 불러오기

In [2]:
import pandas as pd
import geopandas as gpd
from fiona.crs import from_epsg # 좌표변환 

c:\Users\thlee\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


### 데이터 로딩

In [3]:
festival_gdf = gpd.read_file('data/shp/festival_pt.shp')
festival_gdf.head()

,name,organizati,geometry
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031)
1,코로나로 인해 전체 일정 취소,서울특별시 강서구,POINT (942927.290 1952222.192)
2,강동선사문화축제,서울특별시 강동구,POINT (967395.183 1951230.727)
3,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (965465.452 1949694.336)
4,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (964472.438 1949878.657)


In [4]:
subway_gdf = gpd.read_file('data/shp/subway_pt.shp')
subway_gdf.head()

,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,geometry
0,우이신설선,202112,신설동,55769.0,54272.0,110041.0,0.024565,POINT (957902.000 1953075.366)
1,우이신설선,202112,보문,41012.0,41211.0,82223.0,0.018187,POINT (957566.287 1954096.800)
2,우이신설선,202112,성신여대입구(돈암),91571.0,101326.0,192897.0,0.043562,POINT (957317.436 1954894.797)
3,우이신설선,202112,정릉,124880.0,113381.0,238261.0,0.053963,POINT (957048.129 1956079.564)
4,우이신설선,202112,북한산보국문,162606.0,153956.0,316562.0,0.071915,POINT (956599.178 1957073.674)


In [5]:
# 좌표계 확인하는 게 뭐였더라.......? 
festival_gdf.***

SyntaxError: invalid syntax (3847512655.py, line 2)

In [6]:
#서울시 구 행정구역 
gu_gdf = gpd.read_file('data/shp/LARD_ADM_SECT_SGG_11.shp', encoding='cp949')
gu_gdf.head(2)

,ADM_SECT_C,SGG_NM,SGG_OID,COL_ADM_SE,GID,geometry
0,11740,강동구,337,11740,127,"POLYGON ((969713.132 1948846.288, 969692.451 1..."
1,11710,송파구,1969,11710,128,"POLYGON ((968640.372 1944982.492, 968640.957 1..."


## 1. 거리 산출  
1.1 특정 축제 위치에 가장 가까운 지하철역은?  

In [7]:
# 최근린분석 함수
import numpy as np
from scipy.spatial import cKDTree, KDTree
from shapely.geometry import Point

def ckdnearest(gdfA, gdfB):

    nA = np.array(list(gdfA.geometry.apply(lambda x: (x.centroid.x, x.centroid.y))))
    nB = np.array(list(gdfB.geometry.apply(lambda x: (x.centroid.x, x.centroid.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1) 
    gdfB_nearest = gdfB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdfA.reset_index(drop=True),
            gdfB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

In [8]:
# 내가 원하는 하나의 축제 데이터를 추출
festival_gdf2 = festival_gdf[ festival_gdf.name	 == '구로G페스티벌' ]
festival_gdf2

,name,organizati,geometry
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031)


In [9]:
# 위에서 만든 함수를 이용해서 "구로G페스티벌"에서 가장 가까운 지하철역 위치를 찾아보자 
festival_gdf3 = ckdnearest(festival_gdf2, subway_gdf)
festival_gdf3

,name,organizati,geometry,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,dist
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031),2호선,202112,대림(구로구청),645828.0,660728.0,1306556.0,0.298898,638.881443


In [10]:
# 모든 축제에 대해 최근린 분석을 하면
festival_gdf4 = ckdnearest(festival_gdf, subway_gdf)
festival_gdf4.head(10)

,name,organizati,geometry,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,dist
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031),2호선,202112,대림(구로구청),645828.0,660728.0,1306556.0,0.298898,638.881443
1,코로나로 인해 전체 일정 취소,서울특별시 강서구,POINT (942927.290 1952222.192),9호선,202112,가양,544552.0,527052.0,1071604.0,0.245029,691.716009
2,강동선사문화축제,서울특별시 강동구,POINT (967395.183 1951230.727),8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,1108.223457
3,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (965465.452 1949694.336),5호선,202112,광나루(장신대),357309.0,331390.0,688699.0,0.157238,489.585139
4,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (964472.438 1949878.657),5호선,202112,광나루(장신대),357309.0,331390.0,688699.0,0.157238,567.444225
5,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (964663.550 1950386.530),5호선,202112,광나루(장신대),357309.0,331390.0,688699.0,0.157238,823.794072
6,봉황각 3.1독립운동 재현행사,서울특별시 강북구,POINT (956433.939 1962414.146),우이신설선,202112,북한산우이,76806.0,70898.0,147704.0,0.033200,658.800426
7,4.19혁명 국민문화제,서울특별시 강북구,POINT (958132.508 1960073.998),4호선,202112,수유(강북구청),926640.0,926047.0,1852687.0,0.424113,128.169262
8,종교연합바자회,서울특별시 강북구,POINT (957026.073 1959616.863),우이신설선,202112,화계,91530.0,85751.0,177281.0,0.039981,415.148946
9,2022 종로한복축제,서울특별시 종로구,POINT (953408.714 1952497.598),5호선,202112,광화문(세종문화회관),778061.0,807701.0,1585762.0,0.362914,432.243073


이외에도 근린분석에 다양한 패키지(알고리즘)를 활용할 수 있습니다.     
sklearn.metrics.pairwise.euclidean_distances   
shapely.ops.nearest_points    

나라가 다른 도시 위경도 좌표가 주어졌을 때에는 (5179 좌표계 범위를 벗어나는 규모의)    
대원거리(Haversine) 공식을 사용하는 것이 좋습니다.     
https://pypi.org/project/haversine/

1.2 Geopandass의 최근린분석 기능인 sjoin_******* 를 이용해서 각 축제 위치에서 가장 가까운 지하철역과 거리를 산출해 봅시다   
여기를 참조하세요 - tinyurl.com/yvt98ycs 

In [11]:
festival_gdf5 = gpd.sjoin_*******(festival_gdf, subway_gdf, distance_col="distances")
festival_gdf5.head() 

,name,organizati,geometry,index_right,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,distances
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031),342,2호선,202112,대림(구로구청),645828.0,660728.0,1306556.0,0.298898,638.881443
1,코로나로 인해 전체 일정 취소,서울특별시 강서구,POINT (942927.290 1952222.192),53,9호선,202112,가양,544552.0,527052.0,1071604.0,0.245029,691.716009
2,강동선사문화축제,서울특별시 강동구,POINT (967395.183 1951230.727),66,8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,1108.223457
13,제15회 구리한강 유채꽃 축제,경기도 구리시,POINT (967580.935 1952852.498),66,8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,2732.820904
14,제15회 구리 코스모스 축제,경기도 구리시,POINT (967580.935 1952852.498),66,8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,2732.820904


#### sjoin_nearest의 max_distance 파라미터를 이용하여, 각 페스티벌(festival_gdf) 위치에서 반경 800m 이내에 있는 가장 가까운 지하철역들을 찾아봅시다!
max_distance는 PostGIS 등의 공간DBMS의 ST_DWithin(https://postgis.net/docs/ST_DWithin.html) 처럼 임계(제한)거리 이내 조건만 설정할 수 있는 역할을 합니다. 

In [17]:
festival_gdf6 = gpd.sjoin_*******(festival_gdf, subway_gdf, distance_col="distances", ***_*******=***)
festival_gdf6.tail() 

,name,organizati,geometry,index_right,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,distances
38,한성백제문화제,서울특별시 송파구,POINT (966024.797 1946852.454),63,8호선,202112,몽촌토성(평화의문),167322.0,170248.0,337570.0,0.076732,428.686694
39,2020 영양고추 H.O.T Festival,경상북도 영양군,POINT (953885.898 1952011.939),383,1호선,202112,시청,570037.0,569533.0,1139570.0,0.260612,93.523962
42,정월대보름 큰잔치,서울특별시 도봉구,POINT (960171.978 1963569.021),198,경원선,202112,방학,259698.0,253974.0,513672.0,0.117108,512.802337
45,도봉구민 건강축제,서울특별시 도봉구,POINT (960053.782 1963331.194),198,경원선,202112,방학,259698.0,253974.0,513672.0,0.117108,274.888325
46,도봉 과학축전,서울특별시 도봉구,POINT (960053.782 1963331.194),198,경원선,202112,방학,259698.0,253974.0,513672.0,0.117108,274.888325


## 2. 버퍼 생성 및 포함 방식
반경 거리 내에 지하철역이 있는 축제를 찾아봅시다

In [ ]:
# 축제 포인트 지오메트리에서 반경 700m 버퍼 폴리곤 지오메트리를 생성합니다. 
festival_buf_geom = festival_gdf2.geometry.buffer(700)
festival_buf_geom

0    POLYGON ((946627.323 1944204.031, 946623.952 1...
dtype: geometry

In [ ]:
# 축제의 원 속성과 버퍼 폴리곤 지오메트리를 합쳐서 버퍼 폴리곤 GeoDataFrame을 만듭니다. 
festival_buf_gdf = gpd.GeoDataFrame(festival_gdf2[['name']], geometry=festival_buf_geom, crs=from_epsg(5179) )
festival_buf_gdf.info()
festival_buf_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      1 non-null      object  
 1   geometry  1 non-null      geometry
dtypes: geometry(1), object(1)
memory usage: 24.0+ bytes


c:\Users\thlee\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,name,geometry
0,구로G페스티벌,"POLYGON ((946627.323 1944204.031, 946623.952 1..."


In [ ]:
festival_buf_gdf.explore(tiles='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}&s=Ga', attr='google')

In [ ]:
result_gdf = gpd.sjoin(festival_buf_gdf, subway_gdf, how='left', op="포함한다가 뭐였지?")   
result_gdf

C:\Users\thlee\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,name,geometry,index_right,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA
0,구로G페스티벌,"POLYGON ((946627.323 1944204.031, 946623.952 1...",342,2호선,202112,대림(구로구청),645828.0,660728.0,1306556.0,0.298898


#### 서비스로 Geopandas의 sjoin을 쓰지 않는 방식...

In [ ]:
# 축제 위치 포인트를 포함하는 구 폴리곤 찾기 
result = gu_gdf['geometry'].apply(lambda x: festival_gdf['geometry'].within(x))
gu_gdf2 = gu_gdf[(result==True).any(axis=1)]

In [ ]:
gu_gdf2.explore('SGG_NM', categorical=True, tiles='https://a.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png', attr='Carto')

## 수고 많으셨습니다!!!